In [30]:
import math, copy, random, uuid

from cmu_112_graphics import *
from tkinter import *


class GameObject(object):
    def __init__(self, name, **kwargs):
        self.uuid = uuid.uuid4()
        self.name = name
        print('[init][%s][%s] %s' % (self.__class__.__name__, self.uuid, self))
    
    def __repr__(self):
        return self.name


class Scenario(GameObject):
    def __init__(self, name, rows, cols, **kwargs):
        super().__init__(name)
        self.rows = rows
        self.cols = cols
        self.board = [([0] * cols) for i in range(rows)]
        
    def getBorders(self):
        return self.rows, self.cols

class Map(Scenario):
    def __init__(self, name, rows, cols, **kwargs):
        super().__init__(name, rows, cols, **kwargs)
        self.randomMap(p=0.97)
#         self.testMap()
    
    def randomMap(self, p=0.999):
        for i in range(self.rows):
            for j in range(self.cols):
                if random.random() > p:
                    self.board[i][j] = 1
    
    def testMap(self):
        self.board[1][1] = 1
        self.board[2][3] = 1
        self.board[6][9] = 1
        self.board[7][10] = 1
        self.board[7][0] = 1
        self.board[0][10] = 1


class Item(GameObject):
    def __init__(self, **kwargs):
        super().__init__(name)
        self.category = 0


class Creature(GameObject):
    def __init__(self, name, **kwargs):
        super().__init__(name)
        
        self.type = 0
        
        self.level = 1
        self.experience = 0
        
#         self.str = 10
#         self.dex = 10
#         self.agi = 10
#         self.int = 10
#         self.con = 10
#         self.luk = 10
        
        self.hp = 0
        self.sp = 0
        self.attack = 0
        self.defense = 0
        self.hit = 0
        self.avoid = 0
        
        self.posRow = 0
        self.posCol = 0
        
        self.maxDice = 20
    
    def dice(self):
        return random.randint(1, self.maxDice)
    
    def move(self, drow, dcol):
        self.posRow += drow
        self.posCol += dcol
    
    def getPos(self):
        return self.posRow, self.posCol


class Player(Creature):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)


class Monster(Creature):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)


class Game(App):
    def __init__(self, name, width=420, height=420, x=0, y=0, **kwargs):
        super().__init__(width=width, height=height, x=x, y=y, **kwargs)
        # constant
        self.CELL_PIXEL = 10
        self.MAP_PADDING = 20
        
        # game
        self.name = name

        # maps
        self.maps = []
        self.initMaps()
        self.map = self.maps[0]
        
        # slide window frame setting
        # should be odd
        self.slideWindowRows = 7
        self.slideWindowCols = 11
        
        self.offsetRows = self.slideWindowRows // 2
        self.offsetCols = self.slideWindowCols // 2
        
        self.scrollRow = 0
        self.scrollCol = 0
        
        # player
        self.player = None
        self.initPlayer()

    def appendMap(self, o):
        self.maps.append(o)
    
    def initMaps(self):
        self.appendMap(Map('map_test_24_30', 24, 30))
        self.appendMap(Map('map_test_11_15', 11, 15))
    
    def initPlayer(self):
        self.player = Player('Hero')

#     def initMonsters(self, number):
#         self.monsters = []
#         for i in range(1, 6):
#             m = Monster('slime_%d' % i)
#             self.monsters.append(m)
    
    def getMapBorders(self):
        return self.map.getBorders()
    
    def getPlayerPos(self):
        return self.player.getPos()
    
    def movePlayer(self, drow, dcol):
        curRow, curCol = self.getPlayerPos()
        
        tryRow = curRow + drow
        tryCol = curCol + dcol
        
        mapBorderRow, mapBorderCol = self.getMapBorders()
        
        if tryRow >= 0 and tryRow < mapBorderRow and tryCol >= 0 and tryCol < mapBorderCol:
            self.player.move(drow, dcol)
    
    def scrollMap(self):
        curRow, curCol = self.getPlayerPos()
        mapBorderRow, mapBorderCol = self.getMapBorders()
        
        if curRow <= self.offsetRows:
            self.scrollRow = 0
        elif curRow < mapBorderRow - self.slideWindowRows + self.offsetRows:
            self.scrollRow = curRow - self.offsetRows
        else:
            self.scrollRow = mapBorderRow - self.slideWindowRows
        
#         print('offsetcols', offsetCols, 'mapbordercol', mapBorderCol, ' - wincol', self.slideWindowCols)
#         print('scroll', self.scrollCol, 'col', curCol, offsetCols, mapBorderCol - self.slideWindowCols)
        if curCol <= self.offsetCols:
            self.scrollCol = 0
        elif curCol < mapBorderCol - self.slideWindowCols + self.offsetCols:
            self.scrollCol = curCol - self.offsetCols
        else:
            self.scrollCol = mapBorderCol - self.slideWindowCols
        
    def keyPressed(self, event):
        if event.key == 'Down':
            self.movePlayer(+1, 0)
            self.scrollMap()
        elif event.key == 'Left':
            self.movePlayer(0, -1)
            self.scrollMap()
        elif event.key == 'Right':
            self.movePlayer(0, +1)
            self.scrollMap()
        elif event.key == 'Up':
            self.movePlayer(-1, 0)
            self.scrollMap()

    def redrawAll(self, canvas):
        self.drawMap(canvas)
        self.drawPlayer(canvas)
        self.drawText(canvas)

    def drawMap(self, canvas):
        x0 = self.MAP_PADDING
        y0 = self.MAP_PADDING
        x1 = self.slideWindowCols * self.CELL_PIXEL + self.MAP_PADDING
        y1 = self.slideWindowRows * self.CELL_PIXEL + self.MAP_PADDING
        fill = 'white'
        canvas.create_rectangle(x0, y0, x1, y1, fill=fill)
        
#         print('row, col:', self.scrollRow, self.scrollCol)
        for i in range(self.slideWindowRows):
            for j in range(self.slideWindowCols):
                row = i + self.scrollRow
                col = j + self.scrollCol
                if self.map.board[row][col] == 1:
                    x0 = j * self.CELL_PIXEL + self.MAP_PADDING
                    y0 = i * self.CELL_PIXEL + self.MAP_PADDING
                    x1 = (j + 1) * self.CELL_PIXEL + self.MAP_PADDING
                    y1 = (i + 1) * self.CELL_PIXEL + self.MAP_PADDING
                    fill = 'black'
                    canvas.create_oval(x0, y0, x1, y1, fill=fill)
                    canvas.create_text(x1 + 20, (y0 + y1) / 2, text='(%d, %d)' % (row, col), fill=fill, font='Arial 8')

    def drawPlayer(self, canvas):
        curRow, curCol = self.getPlayerPos()
        mapBorderRow, mapBorderCol = self.getMapBorders()
        
        row = self.offsetRows
        if curRow < self.offsetRows:
            row = curRow
        elif curRow > mapBorderRow - self.slideWindowRows + self.offsetRows- 1:
            row = curRow - self.slideWindowRows + self.offsetRows + 1
        
        col = self.offsetCols
        if curCol < self.offsetCols:
            col = curCol
        elif curCol > mapBorderCol - self.slideWindowCols + self.offsetCols - 1:
            col = curCol - self.slideWindowCols + self.offsetCols + 1
        
        x0 = col * self.CELL_PIXEL + self.MAP_PADDING + 1
        y0 = row * self.CELL_PIXEL + self.MAP_PADDING + 1
        x1 = (col + 1) * self.CELL_PIXEL + self.MAP_PADDING - 1
        y1 = (row + 1) * self.CELL_PIXEL + self.MAP_PADDING - 1
        fill = 'grey'
        canvas.create_oval(x0, y0, x1, y1, fill=fill)
    
    def drawText(self, canvas):
#         canvas.create_text(self.width - 100, self.height - 10, text='%s (%d, %d), Map (%d, %d)' % 
#                            (self.player, self.player.posRow, self.player.posCol, self.width, self.height), fill='black', font='Arial 8')
        canvas.create_text(30, self.height - self.MAP_PADDING // 2, text='pos(%d, %d)' % self.getPlayerPos(), fill='black', font='Arial 8')
        canvas.create_text(self.width - 40, self.height - self.MAP_PADDING // 2, text='map(%d, %d)' % self.getMapBorders(), fill='black', font='Arial 8')
        canvas.create_text(self.MAP_PADDING, self.MAP_PADDING // 2, text='%d' % self.scrollCol, fill='black', font='Arial 8')
        canvas.create_text(self.MAP_PADDING // 2, self.MAP_PADDING, text='%d' % self.scrollRow, fill='black', font='Arial 8')

#     def drawMonsters(self, canvas):
#         pass


class GameDelegator(object):
    def __init__(self, name, width=140, height=110, x=750, y=550, **kwargs):
        self.game = Game(name, width, height, x, y, autorun=False, title='Demo')
    
    def run(self):
        self.game.run()


game = GameDelegator('DND', 300, 200, 400, 200)
game.run()

[init][Map][a9fce831-0304-451a-8670-8cfe75e1e3a7] map_test_24_30
[init][Map][ee519582-7914-414a-92ae-6c60bfce148e] map_test_11_15
[init][Player][fdd9026f-41d4-45e5-89a4-b2858ea9b79c] Hero
*** Closing Game 'Demo'.  Bye! ***

